In [1]:
from carmon import registry_pipe

Взглянем на ошибочные отработки

In [2]:
error_stage = registry_pipe.read_input_tables("test_data", 
                                              "test_data",
                                              separator="\t")

In [3]:
error_stage

{'success': False, 'payload': "[Errno 13] Permission denied: 'test_data'"}

In [4]:
error_stage['payload']

"[Errno 13] Permission denied: 'test_data'"

In [5]:
error_stage = registry_pipe.read_input_tables("test_data/tab1.csv", 
                                              "test_data/tab2.csv",
                                              separator="\t")

In [6]:
error_stage

{'success': False,
 'payload': "[Errno 2] No such file or directory: 'test_data/tab1.csv'"}

Теперь проверим корректный запуск, передадим пути к файлам с таблицами

In [7]:
stage_1 = registry_pipe.read_input_tables("test_data/test_registry_pipe/FROM_2.txt", 
                                          "test_data/test_registry_pipe/FROM_3.txt",
                                          separator="\t")

Проверим успех операции

In [8]:
stage_1['success']

True

In [9]:
df = stage_1['payload']
df.head(3)

,litex_sk,litex_value,region
64,90722,5793,Брянская область
65,90723,5701,Брянская область
67,90725,5610,Брянская область


Считываем таблицу со всеми реестрами

In [10]:
stage_2 = registry_pipe.read_all_registry_info("test_data/test_registry_pipe/registry.csv")

In [11]:
stage_2['success']

True

In [12]:
ref = stage_2['payload']
ref.head(3)

,registry_id,depart_name,sample_number,value
0,5263,ФБУЗ ЦГиЭ в Ярославской области,yaro19SM110383,3137
1,5263,ФБУЗ ЦГиЭ в Ярославской области,yaro19SM110415,3136
2,5263,ФБУЗ ЦГиЭ в Ярославской области,yaro19SM110438,3150


Производим подготовительные манипуляции с таблицами

In [13]:
stage_3 = registry_pipe.append_desired_columns(df)

In [14]:
stage_3['success']

True

In [15]:
df = stage_3['payload']
df.head(3)

,litex_sk,litex_value,region,region_code,registry_id,depart_name,sample_number,value,status
64,90722,5793,Брянская область,brya,,,,,
65,90723,5701,Брянская область,brya,,,,,
67,90725,5610,Брянская область,brya,,,,,


Производим поиск образцов в реестре

In [16]:
stage_4 = registry_pipe.process_table_concatenation(df, ref)

In [17]:
stage_4['success']

True

In [18]:
stage_4['payload'].tail(3)  # тут конец таблицы, так как иначе видно личные данные образца

,litex_sk,litex_value,region,region_code,registry_id,depart_name,sample_number,value,status
174,100033,393319,Рязанская область,ryaz,4517,ФБУЗ ЦГиЭ в Рязанской области,ryaz23SM75130,393319,OK
175,100034,374619,Рязанская область,ryaz,4517,ФБУЗ ЦГиЭ в Рязанской области,ryaz23SM74866,374619,OK
176,100035,359319,Рязанская область,ryaz,,,,,NO MATHCES


In [19]:
optional_stage = registry_pipe.save_concatenated_table(stage_4['payload'],
                                                       "test_data/test_registry_pipe/result.csv",
                                                       separator="\t")

In [20]:
optional_stage['success']

True

In [21]:
optional_stage['payload']

'Успешно сохранено в `test_data/test_registry_pipe/result.csv`'

Аналогично для сохранения есть возможность обработки ошибок. 

In [22]:
registry_pipe.save_concatenated_table("nothing", "test_data/test_registry_pipe/result.csv", separator="\t")['payload']

"'str' object has no attribute 'to_csv'"